In [1]:
!pip install pymorphy2[fast]

## Преобразователь новостей в токены

In [2]:
from typing import List
import nltk
import string
import pymorphy2
import codecs

class PrepareNew():
  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
    self.tokenizer = nltk.WordPunctTokenizer()
    self.stopwords = set(line.strip() for line in codecs.open('rus_stopwords.txt', "r", "utf_8_sig").readlines())

  def prepare_corp(self, news_list: List[str]):
    return [self.newstext2token(news_text) for news_text in news_list]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c not in string.punctuation for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords and w.isalpha()]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      return tokens_last


## Usage Example

In [3]:
test_news = [
             "Вышла новая серия Локи",
             "Кофе очень вкусный", 
             "Нил Гейман прекрасен",
             "Металлика топ",
             "Толстой тексты, переполненные водой и своеобразным разжёвыванием, повторением информации для читателя"
]
test_corp = PrepareNew().prepare_corp(test_news)
for i in test_corp:
  print(i)

['выйти', 'новый', 'серия', 'лока']
['кофе', 'вкусный']
['нил', 'гейман', 'прекрасный']
['металлик', 'топ']
['толстой', 'текст', 'переполненный', 'вода', 'своеобразный', 'разжёвывание', 'повторение', 'информация', 'читатель']


## Получение текста ведомостей

In [4]:
raw_data = [line.strip() for line in codecs.open('ved.txt', "r", "utf_8_sig").readlines() if line.strip()!=""]

In [5]:
raw_data[:10]

['Реальный эффективный курс рубля в январе снизился на 3,1%',
 'Реальный эффективный курс рубля к корзине валют государств - основных торговых партнеров России в январе 2015 г. снизился на 3,1% по отношению к декабрю 2014 г.,\xa0говорится в сообщении Центрального банка.Индекс реального курса рубля к доллару понизился на 6,2%, к евро - на 2,1%.Индекс номинального эффективного курса рубля к иностранным валютам снизился на 6,5%, в том числе к евро - на 5,9%, к доллару - на 10,2%.По уточненным данным Банка России, реальный эффективный курс рубля в 2014 г. снизился на 26,7% (по опубликованной ранее предварительной оценке - на 27,2%) против снижения на 2,8% в 2013 г. и укрепления на 5,3% в 2012 г.',
 'Девальвация и контрсанкции обеспечили половину инфляции в 2014 году - Минэкономразвития',
 'Вклад девальвации рубля в инфляцию 2014 г. составил 4,2-4,3 процентных пункта, тогда как вклад продовольственного эмбарго был на уровне 1,4 п. п., отмечается в мониторинге Министерства экономического раз

In [6]:
my_corp = PrepareNew().prepare_corp(raw_data)

In [7]:
my_corp = my_corp[1::2]

## Модель-эмбеддер

In [8]:
from gensim.models import Word2Vec
from gensim.models import Phrases
bigram_transformer = Phrases(my_corp)
model = Word2Vec(bigram_transformer[my_corp], window=5, min_count=2)

In [9]:
model.wv.most_similar('рубль', topn=10)

[('падать', 0.9819367527961731),
 ('цена_продовольствие', 0.977623701095581),
 ('оставаться_низкий', 0.973404049873352),
 ('существенно_увеличиться', 0.9731937646865845),
 ('чистый_отток', 0.9731248021125793),
 ('следующий_десятилетие', 0.9726873636245728),
 ('сильно', 0.9725794792175293),
 ('доходность_облигация', 0.9713980555534363),
 ('говориться_обзор', 0.9708833694458008),
 ('снижаться', 0.9707979559898376)]

In [12]:
model.save("word2vec.model")